In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re

# Movies_Metadata

### Drop NULL's

In [2]:
# read data
df_mm = pd.read_csv('../data/raw/movies_metadata.csv',low_memory=False)
df_mm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
# check null values in percentage
print(df_mm.isna().sum(axis=0)/len(df_mm))
# drop the columns with too many null values and the irrelevant columns
df_mm = df_mm.drop(columns=['belongs_to_collection','homepage','tagline',
                            'poster_path'])

adult                    0.000000
belongs_to_collection    0.901157
budget                   0.000000
genres                   0.000000
homepage                 0.828839
id                       0.000000
imdb_id                  0.000374
original_language        0.000242
original_title           0.000000
overview                 0.020983
popularity               0.000110
poster_path              0.008490
production_companies     0.000066
production_countries     0.000066
release_date             0.001914
revenue                  0.000132
runtime                  0.005785
spoken_languages         0.000132
status                   0.001914
tagline                  0.551049
title                    0.000132
video                    0.000132
vote_average             0.000132
vote_count               0.000132
dtype: float64


In [4]:
# check null values in each feature
print("empty values for genres:", (df_mm['genres'] == '[]').sum())
print("empty values for production_companies:", (df_mm['production_companies'] == '[]').sum())
print("empty values for production_countries:", (df_mm['production_countries'] == '[]').sum())
print("empty values for spoken_languages:", (df_mm['spoken_languages'] == '[]').sum())

empty values for genres: 2442
empty values for production_companies: 11875
empty values for production_countries: 6282
empty values for spoken_languages: 3829


### Convert JSON to list

In [5]:
def Spok_Lang(x):
    '''
    For spoken_languages feature:
    Input a string of JSON, get the values for which the
    keys are "iso_639_1" (stands for country code) then 
    convert the string into a list
    '''
    if ((x is np.nan) or (x=='[]')):
        return np.nan
    
    # would have invalid escape error
    x = re.sub(r'\\','',x)
    x_json = json.loads(x.replace("\'","\""))
    return [i['iso_639_1'] for i in x_json] 

df_mm['spoken_languages'] = df_mm['spoken_languages'].apply(Spok_Lang)

In [6]:
def Gen(x):
    '''
    For genres feature:
    Input a string of JSON, get the values for which the
    keys are "name" then convert the string into a list
    '''
    if ((x is np.nan) or (x=='[]')):
        return np.nan
    
    x_json = json.loads(x.replace("\'","\""))
    return [i['name'] for i in x_json]

df_mm['genres'] = df_mm['genres'].apply(Gen)

In [7]:
def Prod_Count(x):
    '''
    For production_countries feature:
    Input a string of JSON, get the values for which the
    keys are "iso_639_1" (stands for country code) then 
    convert the string into a list
    '''
    if ((x is np.nan) or (x=='[]')):
        return np.nan
    
    try:
        # some observations have ill-formatted values just drop them
        x_json = json.loads(x.replace("\'","\""))
        x_list = [i['iso_3166_1'] for i in x_json]
    except:
        return np.nan
    
    return x_list

df_mm['production_countries'] = df_mm['production_countries'].apply(Prod_Count)

In [8]:
def Prod_Com(x):
    '''
    For production_companies feature:
    Input a string of JSON, get the values for which the
    keys are "name" then convert the string into a list
    '''
    if ((x is np.nan) or (x=='[]')):
        return np.nan
    
    try:
        # some observations have ill-formatted values just drop them
        x_json = json.loads(x.replace("\'","\""))
        x_list = [i['name'] for i in x_json]
    except:
        return np.nan
    
    return x_list

df_mm['production_companies'] = df_mm['production_companies'].apply(Prod_Com)

### Convert Other Features

In [9]:
# convert the date string to datetime
df_mm['release_date'] = pd.to_datetime(df_mm['release_date'],errors='coerce')

# convert to boolean values
df_mm['adult'] = df_mm['adult'].apply(lambda x: True if x == 'True' else False)

In [10]:
def ConvertToFloat(x):
    '''
    for feature budget: convert the string
    into float. If the input is ill-formatted,
    then return Nan
    '''
    try:
        float(x)
    except:
        return np.nan
    return float(x)

df_mm['budget'] = df_mm['budget'].apply(ConvertToFloat)
df_mm['popularity'] = df_mm['popularity'].apply(ConvertToFloat)

### Combine into One Function

In [11]:
# print the null values after cleaning
df_mm.isna().sum()

adult                       0
budget                      3
genres                   2442
id                          0
imdb_id                    17
original_language          11
original_title              0
overview                  954
popularity                  6
production_companies    12365
production_countries     6293
release_date               90
revenue                     6
runtime                   263
spoken_languages         3835
status                     87
title                       6
video                       6
vote_average                6
vote_count                  6
dtype: int64

In [12]:
def PreprocessMM(inpath,outpath,save=False):
    '''
    clean the raw movies_metadata.csv dataset
    
    Params:
        path: path of the input dataset(Ex. '../data/raw/movies_metadata.csv')
        save: specify if the cleaned dataset 
              need to be saved in the '../data/interim/'
    '''
    # read data
    filename = "movies_metadata.csv"
    df_mm = pd.read_csv(inpath+filename,low_memory=False)
    # drop columns
    df_mm = df_mm.drop(columns=['belongs_to_collection','homepage','tagline','poster_path'])
    
    # convert JSON in string to list
    df_mm['spoken_languages'] = df_mm['spoken_languages'].apply(Spok_Lang)
    df_mm['genres'] = df_mm['genres'].apply(Gen)
    df_mm['production_countries'] = df_mm['production_countries'].apply(Prod_Count)
    df_mm['production_companies'] = df_mm['production_companies'].apply(Prod_Com)
    
    # convert the date string to datetime
    df_mm['release_date'] = pd.to_datetime(df_mm['release_date'],errors='coerce')
    # convert to boolean values
    df_mm['adult'] = df_mm['adult'].apply(lambda x: True if x == 'True' else False)
    # convert from string to float
    df_mm['budget'] = df_mm['budget'].apply(ConvertToFloat)
    df_mm['popularity'] = df_mm['popularity'].apply(ConvertToFloat)
    
    if save:
        filename = "movies_metadata_clean.csv"
        df_mm.to_csv(outpath+filename)
        print("the cleaned dataset of movies_metadata is saved in {}".format(outpath))
    
    return df_mm

# Credits

### Convert JSON to list

In [13]:
# read data
df_c = pd.read_csv("../data/raw/credits.csv")
# print nan
df_c.isna().sum()
# print empty vector
print("empty values in cast:",(df_c['cast'] == '[]').sum())
print("empty values in crew:",(df_c['crew'] == '[]').sum())

empty values in cast: 2418
empty values in crew: 771


In [14]:
def Cas(X):
    '''
    For cast feature:
    Input a string of JSON, get the values for which the
    keys are "name"(name of actors) then convert the string into a list
    '''
    # remove some invalid char
    x = re.sub("\"[^\"]*\",","null,",X)
    x = re.sub("\"[^\"]*\"}","null}",x)
    x = x.replace("\"", "")
    x = x.replace("\'","\"").replace("None","null")
    # replace the character into null
    x = re.sub(r'\\','',x)
    x = re.sub("\"character\":\s.+?(?=,\s\")","\"character\": null",x)
    
    try:
        x_json = json.loads(x)
    except:
        return
    
    x_list = [i['name'] for i in x_json]
    
    if len(x_list) == 0:
        return
    return x_list

df_c['cast'] = df_c['cast'].apply(Cas)

In [15]:
def Cre(x):
    '''
    For crew feature:
    Input a string of JSON, get the values for which the
    keys are "name"(name of directors) then convert the string into a list
    '''
    x = re.sub("\"[^\"]*\",","null,",x)
    x = x.replace("\"", "")
    x = x.replace("\'","\"").replace("None","null")
    x = re.sub(r'\\','',x)
    x_json = json.loads(x)
    x_list = [i['name'] for i in x_json if i['job'] == 'Director' ]
    
    if len(x_list) == 0:
        return 
    return x_list

df_c['crew'] = df_c['crew'].apply(Cre)

### Combine Into One Function

In [16]:
# print the null values after cleaning
df_c.isna().sum()

cast    2420
crew     887
id         0
dtype: int64

In [17]:
def PreprocessC(inpath,outpath,save=False):
    '''
    clean the raw credits.csv dataset
    
    Params:
        path: path of the input dataset(Ex. '../data/raw/credits.csv')
        save: specify if the cleaned dataset 
              need to be saved in the '../data/interim/'
    '''
    # read data
    filename = "credits.csv"
    df_c = pd.read_csv(inpath+filename)
    
    # convert the json data
    df_c['cast'] = df_c['cast'].apply(Cas)
    df_c['crew'] = df_c['crew'].apply(Cre)
    
    if save:
        filename = "credits_clean.csv"
        df_c.to_csv(outpath+filename)
        print("the cleaned dataset of credits is saved in {}".format(outpath))
    
    return df_c

# Keywords

In [18]:
# read data
df_k = pd.read_csv("../data/raw/keywords.csv")
# print nan
df_k.isna().sum()
# the null values in keywords
print("empty values in keywords:", (df_k['keywords'] == "[]").sum())

empty values in keywords: 14795


In [19]:
def Keyw(x):
    x = re.sub("\"[^\"]*\",","null,",x)
    x = re.sub("\"[^\"]*\"}","null}",x)
    x = x.replace("\"", "")
    x = x.replace("\'","\"").replace("None","null")
    x = re.sub(r'\\','',x)
    
    try:
        x_json = json.loads(x)
    except:
        return
    
    x_list = [i['name'] for i in x_json]
    if len(x_list) == 0:
        return 
    
    return x_list

df_k['keywords'] = df_k['keywords'].apply(Keyw)

### Combine Function

In [20]:
# check null values
df_k.isna().sum()

id              0
keywords    14795
dtype: int64

In [21]:
def PreprocessK(inpath,outpath,save=False):
    '''
    clean the raw keywords.csv dataset
    
    Params:
        path: path of the input dataset(Ex. '../data/raw/keywords.csv')
        save: specify if the cleaned dataset 
              need to be saved in the '../data/interim/'
    '''
    # read data
    filename = "keywords.csv"
    df_k = pd.read_csv(inpath+filename)
    
    # convert the json data
    df_k['keywords'] = df_k['keywords'].apply(Keyw)
    
    if save:
        filename = "keywords_clean.csv"
        df_k.to_csv(outpath+filename)
        print("the cleaned dataset of keywords is saved in {}".format(outpath))
    
    return df_k

# Combine

In [26]:
def combine(df_mm,df_c,df_k):
    # convert the type of id to numeric format
    df_mm['id'] = pd.to_numeric(df_mm['id'],errors='coerce')
    df_mm = df_mm.dropna(subset=['id'])
    
    # merge three datasets
    df_comb = pd.merge(df_c,df_k,on='id',how='inner')
    df_comb = pd.merge(df_comb, df_mm,on='id',how='inner')
    
    # drop pulicates and reset index
    df_comb = df_comb.drop_duplicates(subset=['id']).reset_index(drop=True)
    
    return df_comb

In [27]:
combine(df_mm, df_c, df_k)

,cast,crew,id,keywords,adult,budget,genres,imdb_id,original_language,original_title,...,production_countries,release_date,revenue,runtime,spoken_languages,status,title,video,vote_average,vote_count
0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",[John Lasseter],862,"[jealousy, toy, boy, friendship, friends, riva...",False,30000000.0,"[Animation, Comedy, Family]",tt0114709,en,Toy Story,...,[US],1995-10-30,373554033.0,81.0,[en],Released,Toy Story,False,7.7,5415.0
1,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],8844,"[board game, disappearance, None, new home, re...",False,65000000.0,"[Adventure, Fantasy, Family]",tt0113497,en,Jumanji,...,[US],1995-12-15,262797249.0,104.0,"[en, fr]",Released,Jumanji,False,6.9,2413.0
2,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",[Howard Deutch],15602,"[fishing, best friend, duringcreditsstinger, o...",False,0.0,"[Romance, Comedy]",tt0113228,en,Grumpier Old Men,...,[US],1995-12-22,0.0,101.0,[en],Released,Grumpier Old Men,False,6.5,92.0
3,"[Whitney Houston, Angela Bassett, Loretta Devi...",[Forest Whitaker],31357,"[based on novel, interracial relationship, sin...",False,16000000.0,"[Comedy, Drama, Romance]",tt0114885,en,Waiting to Exhale,...,[US],1995-12-22,81452156.0,127.0,[en],Released,Waiting to Exhale,False,6.1,34.0
4,"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Charles Shyer],11862,"[baby, midlife crisis, confidence, aging, daug...",False,0.0,[Comedy],tt0113041,en,Father of the Bride Part II,...,[US],1995-02-10,76578911.0,106.0,[en],Released,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45427,"[Leila Hatami, Kourosh Tahami, Elham Korda]",[Hamid Nematollah],439050,[tragic love],False,0.0,"[Drama, Family]",tt6209470,fa,رگ خواب,...,[IR],NaT,0.0,90.0,[fa],Released,Subdue,False,4.0,1.0
45428,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...",[Lav Diaz],111109,"[artist, play, pinoy]",False,0.0,[Drama],tt2028550,tl,Siglo ng Pagluluwal,...,[PH],2011-11-17,0.0,360.0,[tl],Released,Century of Birthing,False,9.0,3.0
45429,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",[Mark L. Lester],67758,None,False,0.0,"[Action, Drama, Thriller]",tt0303758,en,Betrayal,...,[US],2003-08-01,0.0,90.0,[en],Released,Betrayal,False,3.8,6.0
45430,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",[Yakov Protazanov],227506,None,False,0.0,NaN,tt0008536,en,Satana likuyushchiy,...,[RU],1917-10-21,0.0,87.0,NaN,Released,Satan Triumphant,False,0.0,0.0
